# PyKX Example
Query tick data thru gateway from pyKX

**WORK IN PROGRESS, NOT COMPLETED**

In [ ]:
import boto3
import json
import datetime

import pykx as kx

from managed_kx import *
from env_2 import *

clusters = {
    "rdb_conn": "RDB_basictickdb_20230601",
    "hdb_conn": "HDB_basictickdb_20230601",
    "gw_conn" : "GATEWAY_20230601"
}

# where is the license from?
os.getenv("QHOME")


In [ ]:
# triggers credential get
session=None

try:
    # aws: use ada for credentials
    subprocess.call(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

In [ ]:
conns = {}

for k, v in clusters.items():
    try:
        conn_str = get_kx_connection_string(client, environmentId=ENV_ID, clusterName=v, userName=KDB_USERNAME, boto_session=session)
    except:
        conn_str = ""

    conns[k] = conn_str
    

In [ ]:
conns

# Connect and Query

In [ ]:
hdb_name = clusters['hdb_conn']

# Query the HDB
hdb = get_pykx_connection(client, 
                          environmentId=ENV_ID, clusterName=hdb_name, 
                          userName=KDB_USERNAME, boto_session=session)

In [ ]:
hdb("tables[]").py()

In [ ]:
# Tables
tables = hdb("tables[]").py()
print(f"Tables ({len(tables)}): {tables}")

# Schema
schema_pdf = hdb("meta `example").pd()
display(schema_pdf)


In [ ]:
# Simple Query
res_table = hdb("select counts:count i, avg_num: avg number by date from example").pd()
display(res_table)

# Number of Rows
rows = hdb("count example").py()
print(f"Rows: {rows:,}")